# WCA Results - Indicative Rankings

Created by Michael George (AKA Logiqx)

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [1]:
from EventsLib import *

## Read Results from CSV

Read event data from CSV into memory, prior to processing

In [2]:
import os, csv

class Person:
    
    def __init__(self, fields):
        """Initialisise the person"""
        
        self.id = fields[0]
        self.name = fields[1]
        self.country = fields[2]
        self.username = fields[3]
        self.usernum = int(fields[4])
        self.ageCategory = int(fields[5])
        self.highlight = fields[6]
        if self.highlight == 'Y':
            self.name = '* ' + self.name + ' *'


    def getWcaLink(self, eventId = None):
        if (eventId):
            link = '<a href="https://www.worldcubeassociation.org/persons/%s#%s">%s</a>' % (self.id, eventId, self.name)
        else:
            link = '<a href="https://www.worldcubeassociation.org/persons/%s">%s</a>' % (self.id, self.name)

        return link


    def getSpeedsolvingLink(self):
        if self.usernum > 0:
            profile = '%s.%d' % (self.username.replace(' ', '-').replace('.', '-').lower(), self.usernum)
            link = '<a href="https://www.speedsolving.com/members/%s">%s</a>' % (profile, self.username)
        else:
            link = ''

        return link


class IndicativeResults:
    
    def __init__(self, ageCategory):
        """Initialisise the indicative results"""
        
        self.event = None
        self.persons = {}
        self.ids = []
        self.results = {}
        self.ageCategory = ageCategory


    def readPersons(self, basename):
        """Read seniors from CSV into memory"""
        
        self.persons = {}

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', basename + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for line in csvReader:
                person = Person(line)

                if person.ageCategory >= self.ageCategory and person.id not in self.persons:
                    self.persons[person.id] = person
                    self.ids.append(person.id)

                
    def listPersons(self):
        """List seniors from memory"""
        
        html = '<details id="persons">\n'
        html += '  <summary>%s</summary>\n' % 'Known Over %ds' % self.ageCategory
        html += '  <table>\n'
        html += '    <tr>'
        for field in ['Person', 'Speedsolving.com']:
            html += '<td><b>%s</b></td>' % field
        html += '</tr>\n'
            
        for id in self.ids:
            person = self.persons[id]
            ageSuffix = ', %d+' % int(person.ageCategory) if int(person.ageCategory) > self.ageCategory else ''

            html += '    <tr>'
            html += '<td>%s, %s%s</td>' % (person.getWcaLink(), person.country, ageSuffix)
            html += '<td>%s</td>' % person.getSpeedsolvingLink()
            html += '</tr>\n'

        html += '  </table>\n'
        html += '</details>\n\n'
        
        return html
            
        
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.event = event
        self.results = []
        
        if event[0] == '333fm' and 'single' in basename:
            self.fmSingle = True
        else:
            self.fmSingle = False

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', basename, event[0] + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for inputRow in csvReader:             
                self.results.append(inputRow)
                

    def listResults(self, section):
        """List seniors from memory"""
        
        html = ''
        
        if len(self.results) > 0:
            html = '<details id="%s_%s">\n' % (self.event[0], section)
            html += '  <summary>%s</summary>\n' % self.event[1]
            html += '  <table>\n'
            html += '    <tr>'
            for field in ['Rank', 'Person', 'Result']:
                html += '<td><b>%s</b></td>' % field
            html += '</tr>\n'

            prevResult = None

            for result in self.results:
                rank = result[0]
                person = self.persons[result[1]]
                ageSuffix = ', %d+' % int(result[3]) if int(result[3]) > self.ageCategory else ''

                if self.fmSingle:
                    result = formatResult(self.event, int(result[2]) * 100, showFractions = False)
                else:
                    result = formatResult(self.event, result[2], showFractions = True)

                html += '    <tr>'
                html += '<td style="text-align:center">%s</td>' % (rank if result != prevResult else '')
                html += '<td>%s, %s%s</td>' % (person.getWcaLink(self.event[0]), person.country, ageSuffix)
                html += '<td style="text-align:right">%s</td>' % result
                html += '</tr>\n'

                prevResult = result

            html += '  </table>\n'
            html += '</details>\n\n'
        
        return html

## Analyse Events

Process the events one-by-one

In [3]:
import datetime

def runReport(ageCategory):
    
    fn = 'Indicative_Rankings_%d.md' % ageCategory if ageCategory > 40 else 'Indicative_Rankings.md'
    with open(os.path.join('..', 'templates', fn), 'r') as f:
        html = ''.join(f.readlines())

    refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')
    html += 'Last refreshed: ' + refreshed + ' (UTC)\n\n'

    indicativeResults = IndicativeResults(ageCategory)

    html += '<h2 id="competitors">%s</h2>\n\n' % 'Official Competitors'
    fn = 'known_senior_details'
    indicativeResults.readPersons(fn)
    html += indicativeResults.listPersons()

    ids = indicativeResults.ids
    ids.sort()
    idsCsv = ','.join(id for id in ids)
    html = html.replace("wcaids=", "wcaids=" + idsCsv)

    html += '<h2 id="averages">%s</h2>\n\n' % 'Official Averages'
    for event in events:
        if event[3] != 'multi':
            fn = 'senior_averages'
            indicativeResults.readResults(fn, event)
            html += indicativeResults.listResults("avg")

    fn = 'Indicative_Rankings_%d.md' % ageCategory if ageCategory > 40 else 'Indicative_Rankings.md'
    with open(os.path.join('..', 'docs', fn), 'w') as f:
        f.write(html)

        
runReport(ageCategory = 40)

print('Indicative Rankings updated!')

Indicative Rankings updated!


## All Done!